In [1]:
exec(open('init_notebook.py').read())
from sympy import Polygon, Point2D
import cv2
import matplotlib.pyplot as plt
import pandas as pd

current working dirc:\Users\abjaw\Documents\GitHub\drone-dataset-tools\src\highD_code


In [2]:
dataDir = "C:/Users/abjaw/Documents/GitHub/drone-dataset-tools/data/highD_dataset"
outputDir = "C:/Users/abjaw/Documents/GitHub/drone-dataset-tools/output"
dataset_id = 1
path_dict = get_path_dict(dataDir)
current_file = read_highD_data(dataset_id, path_dict, outputDir)

# print(current_file)
# img = current_file.draw_frame_with_id(1)
# plt.figure(figsize = (25,2))
# plt.imshow(img, interpolation='nearest')

# current_file.create_video_from_frames('output', 1, 1000, 25)

# print(current_file.tracks.columns)

# df = current_file.tracks
# df.columns
# df.loc[(df["xVelocity"]) < 0]



collecting data for highway with ID  1


In [3]:
meta_data = current_file.tracksMeta_dict
data = current_file.track_dict
print('tracks meta ', meta_data[1].keys())
print('tracks ', data[1].keys())

tracks meta  dict_keys(['id', 'width', 'height', 'initialFrame', 'finalFrame', 'numFrames', 'class', 'drivingDirection', 'traveledDistance', 'minXVelocity', 'maxXVelocity', 'meanXVelocity', 'minDHW', 'minTHW', 'minTTC', 'numLaneChanges'])
tracks  dict_keys(['id', 'frame', 'bbox', 'xVelocity', 'yVelocity', 'xAcceleration', 'yAcceleration', 'frontSightDistance', 'backSightDistance', 'thw', 'ttc', 'dhw', 'precedingXVelocity', 'precedingId', 'followingId', 'leftFollowingId', 'leftAlongsideId', 'leftPrecedingId', 'rightFollowingId', 'rightAlongsideId', 'rightPrecedingId', 'laneId'])


In [ ]:
# print('len ', len(current_file.find_initial_state()))
# current_file.find_initial_state()
# import copy

# current_file.create_video_car_follow_maneuver()



In [4]:
current_file.tracksMeta.columns
df = current_file.tracksMeta

car_follow = current_file.filter_car_following()

for i in range(len(car_follow)):
    ego_id = car_follow[i]['ego_id']
    pred_id = car_follow[i]['pred_id']

    video_name = 'car_follow_' + str(ego_id)

    start = car_follow[i]['following_start']
    end = car_follow[i]['following_end']

    print(video_name, start, end)
    current_file.create_video_from_frames(start, end, 25, video_name, ego_id)




car_follow_2 2 29
video name is  2
creating video with name  2.avi


error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'VideoWriter'
> Overload resolution failed:
>  - Argument 'fps' can not be treated as a double
>  - VideoWriter() missing required argument 'frameSize' (pos 5)
>  - VideoWriter() missing required argument 'params' (pos 5)
>  - VideoWriter() missing required argument 'frameSize' (pos 5)


In [ ]:
df.head()

In [ ]:
current_file.tracks.columns

In [ ]:
import statistics
import numpy as np


def find_car_following(meta_data, data, my_type, preceding_type):
    """
    Find out car-following situations from specific vehicle types

    :param meta_data: track meta information from highD dataset
    :param data: track data from highD dataset
    :param my_type: the ego vehicle's type
    :param preceding_type: the preceding vehicle's type

    :return: a list of dictionary with
        ego_id: identification of the ego vehicle
        pred_id: identification of the preceding vehicle
        following_start: the frame when the following starts
        following_duration: the following duration
        following_end: the frame when the following ends
        dhw: array of DHW while following
        thw: array of THW while following
        ttc: array of TTC while following
    """
    following_data = []
    # for each agent (car/truck)
    for i in range(0, len(data)):
        following_started = False
        following_dhw = []
        following_thw = []
        following_ttc = []
        ego_speed = []
        pred_speed = []
        frame_following_start = 0
        following_duration = 0
        preceding_id = 0
        ego_id = data[i].get('id')
        # search in each frame
        for frame in range(0, len(data[i].get('frame'))):

            # if thre is a vehicle in front of the target vehicle
            if data[i].get('precedingId')[frame] != 0:

                # if the preceding vehicle id is not zero and the preceding vehicle changed and following started true
                if preceding_id != 0 and data[i].get('precedingId')[frame] != preceding_id and following_started:
                    frame_following_end = frame - 1
                    # check for longer scenarios???
                    if frame_following_end != frame_following_start:
                        following_data.append(
                            {"ego_id": ego_id, "pred_id": preceding_id, "following_start": frame_following_start,
                             "following_duration": following_duration, "following_end": frame_following_end,
                             "dhw": statistics.mean(following_dhw), "thw": np.nanmean(following_thw),
                             "ttc": statistics.mean(following_ttc),
                             "ego_speed": ego_speed, "pred_speed": pred_speed})
                    following_started = False
                    following_dhw = []
                    following_thw = []
                    following_ttc = []
                    ego_speed = []
                    pred_speed = []
                    frame_following_start = 0
                    following_duration = 0
                # assign preceding vehicle id
                preceding_id = data[i].get('precedingId')[frame]
                if ((get_vehicle_class(meta_data, preceding_id) == preceding_type)
                        and (get_vehicle_class(meta_data, ego_id) == my_type)):
                    if following_started is False:
                        following_started = True
                        frame_following_start = frame
                        if data[i].get('dhw')[frame] > 0:
                            following_dhw.append(data[i].get('dhw')[frame])
                            following_ttc.append(get_ttc(data[i].get('dhw')[frame],
                                                         data[i].get('xVelocity')[frame],
                                                         data[i].get('precedingXVelocity')[frame]))
                        if data[i].get('thw')[frame] > 0:
                            following_thw.append(data[i].get('thw')[frame])
                        else:
                            following_thw.append(np.nan)
                        ego_speed.append(data[i].get('xVelocity')[frame])
                        pred_speed.append(data[i].get('precedingXVelocity')[frame])
                    else:
                        if data[i].get('dhw')[frame] > 0:
                            following_dhw.append(data[i].get('dhw')[frame])
                            following_ttc.append(get_ttc(data[i].get('dhw')[frame],
                                                         data[i].get('xVelocity')[frame],
                                                         data[i].get('precedingXVelocity')[frame]))
                        if data[i].get('thw')[frame] > 0:
                            following_thw.append(data[i].get('thw')[frame])
                        else:
                            following_thw.append(np.nan)
                        ego_speed.append(data[i].get('xVelocity')[frame])
                        pred_speed.append(data[i].get('precedingXVelocity')[frame])
                    following_duration = following_duration + 1
            else:
                if following_started is True:
                    frame_following_end = frame-1
                    if frame_following_end != frame_following_start:
                        following_data.append(
                            {"ego_id": ego_id, "pred_id": preceding_id, "following_start": frame_following_start,
                             "following_duration": following_duration, "following_end": frame_following_end,
                             "dhw": statistics.mean(following_dhw), "thw": np.nanmean(following_thw),
                             "ttc": statistics.mean(following_ttc),
                             "ego_speed": ego_speed, "pred_speed": pred_speed})
                    following_started = False
                    following_dhw = []
                    following_thw = []
                    following_ttc = []
                    ego_speed = []
                    pred_speed = []
                    frame_following_start = 0
                    following_duration = 0
                    preceding_id = 0
    return following_data


def get_vehicle_class(meta_data, id):
    """

    :param meta_data: track meta information from highD dataset
    :param id: vehicle's id in the dataset
    :return: vehicle type (string: 'Car' or 'Truck')
    """
    return meta_data[id].get('class')


def get_ttc(dhw, my_speed, pred_speed):
    my_speed = abs(my_speed)
    pred_speed = abs(pred_speed)
    if my_speed > pred_speed and dhw != 0:
        return dhw / (my_speed - pred_speed)
    else:
        return np.nan


In [ ]:
# all_frame = all_frame[0:2]

# for frame in all_frame:
#     # print(frame)
#     df_frame = df[df['frame'] == frame]
#     polygon_list = []
#     df_bbox = df_frame[['x', 'y', 'width', 'height']]
#     for index, row in df_bbox.iterrows():
#         x = row['x']
#         y = row['y']
#         width = row['width']
#         height = row['height']
#         polygon_list.append(Polygon(Point2D(x, y), Point2D(x+width, y), Point2D(x+width, y+height), Point2D(x, y+height)))
#     is_intersecting = False
#     for i in range(len(polygon_list)):
#         for j in range(i+1, len(polygon_list)):
#             if polygon_list[i].intersection(polygon_list[j]):
#                 is_intersecting = True
#                 break
#         if is_intersecting:
#             print('frame {} is intersecting'.format(frame))
#             break
    
    

# # print(all_frame)

# # df.head(10)
